In [6]:
import sys
import pandas as pd
import numpy as np
#from google.colab import drive 
#!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
#!unzip data.zip
data = pd.read_csv('D:/ML-regression/train.csv', header = None, encoding = 'big5')
#data = pd.read_csv('./train.csv', encoding = 'big5')
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
1,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
2,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
3,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
4,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08


In [7]:
import pandas as pd
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()


In [15]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
#month_data

In [11]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

[[0.  1.  2.  ... 2.5 2.  0.6]
 [1.  2.  3.  ... 2.  0.6 0.8]
 [2.  3.  4.  ... 0.6 0.8 1.6]
 ...
 [0.8 1.8 2.6 ... 1.5 1.  1.7]
 [1.8 2.6 1.4 ... 1.  1.7 1.5]
 [2.6 1.4 1.7 ... 1.7 1.5 2. ]]
[[38.]
 [56.]
 [64.]
 ...
 [50.]
 [32.]
 [22.]]


In [16]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[-1.32940065, -0.57052916,  0.18849156, ...,  0.18190364,
        -0.28611588, -1.59862324],
       [-0.57043373,  0.18838607,  0.9474531 , ..., -0.28631722,
        -1.59712273, -1.41122474],
       [ 0.18853318,  0.9473013 ,  1.70641464, ..., -1.59733562,
        -1.40983604, -0.66163075],
       ...,
       [-0.72222712,  0.03660302,  0.64386849, ..., -0.75453808,
        -1.22254934, -0.5679315 ],
       [ 0.0367398 ,  0.64373521, -0.26688536, ..., -1.22275893,
        -0.56704592, -0.75533   ],
       [ 0.64391333, -0.26696307, -0.0391969 , ..., -0.56724973,
        -0.75433261, -0.28683376]])

In [17]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.32940065 -0.57052916  0.18849156 ...  0.18190364 -0.28611588
  -1.59862324]
 [-0.57043373  0.18838607  0.9474531  ... -0.28631722 -1.59712273
  -1.41122474]
 [ 0.18853318  0.9473013   1.70641464 ... -1.59733562 -1.40983604
  -0.66163075]
 ...
 [ 1.32698356  0.87140977  1.9341031  ...  0.93105701 -1.128906
  -0.9427285 ]
 [ 0.87160341  1.93389109  2.08589541 ... -1.12911476 -0.9416193
  -1.03642775]
 [ 1.93415709  2.08567414  2.61716849 ... -0.94182642 -1.03526265
  -0.84902925]]
[[38.]
 [56.]
 [64.]
 ...
 [22.]
 [22.]
 [24.]]
[[ 2.08595047  2.61691479  1.40283003 ... -1.03547059 -0.84797596
  -1.03642775]
 [ 2.61722732  1.40265043  1.09924541 ... -0.84818225 -1.03526265
   1.39975272]
 [ 1.40288025  1.09908434  0.71976464 ... -1.03547059  1.39946436
  -0.84902925]
 ...
 [-0.72222712  0.03660302  0.64386849 ... -0.75453808 -1.22254934
  -0.5679315 ]
 [ 0.0367398   0.64373521 -0.26688536 ... -1.22275893 -0.56704592
  -0.75533   ]
 [ 0.64391333 -0.26696307 -0.0391969  ... -0.56724973

In [18]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

0:50.16931699765628
100:18.357407699963854
200:15.386607093791305
300:14.036204308826726
400:13.207479473449556
500:12.605841583597607
600:12.135710184253265
700:11.756603076756761
800:11.446093643239942
900:11.189347838078007


array([[ 4.27639774e+01],
       [ 2.32407535e-01],
       [-4.03044984e-01],
       [ 2.82433838e-01],
       [-1.45897758e-01],
       [-3.30022848e-01],
       [ 1.87020781e-01],
       [-2.61732040e-01],
       [ 6.99545300e-01],
       [-4.86140103e-01],
       [ 2.64554581e+01],
       [-4.94123183e+01],
       [ 2.16576940e+01],
       [ 3.22824423e+00],
       [ 1.31809426e+00],
       [-5.45641663e+00],
       [-1.17911084e+00],
       [-7.83841331e+00],
       [ 1.20296088e+01],
       [-5.60295772e-01],
       [ 2.45034207e-01],
       [ 1.69949374e-01],
       [ 4.55884113e-01],
       [-5.01774771e-01],
       [-1.41786538e-01],
       [ 3.36248815e-01],
       [-1.18516703e-01],
       [ 2.07474760e-01],
       [-1.63116945e-01],
       [ 4.23763397e-02],
       [-2.76398058e-01],
       [-2.39708786e-01],
       [-1.06704168e-01],
       [ 6.84539614e-02],
       [-1.99919951e-01],
       [ 2.81544194e-01],
       [ 4.49488420e-01],
       [-4.24377432e-01],
       [ 1.4

In [19]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('D:/ML-regression/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , 14.6089046 , 14.60777537, ..., -1.22275893,
        -1.59712273, -0.47423225],
       [ 1.        ,  9.29613618,  8.53645356, ..., -0.37996139,
        -0.66068927, -1.03642775],
       [ 1.        , 22.95754068, 22.19692764, ..., -1.87826813,
        -1.59712273, -1.13012699],
       ...,
       [ 1.        , 17.64477226, 18.4023515 , ..., -0.28631722,
        -0.75433261,  0.65015873],
       [ 1.        ,  7.01923543,  7.01862311, ..., -1.59733562,
        -1.69076608, -1.69232249],
       [ 1.        ,  9.29613618,  9.29536879, ...,  2.42936375,
         2.71047121,  1.21235422]])

In [20]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 220.85239735],
       [ 318.78482367],
       [ 633.59196739],
       [ 274.72185656],
       [ 589.49811955],
       [ 312.91782197],
       [ 122.9915125 ],
       [ -55.15581024],
       [ 744.24903158],
       [ 794.95951473],
       [-277.34019766],
       [ 623.23898475],
       [1581.78739787],
       [ 601.03975301],
       [ 574.31329378],
       [ 285.35413388],
       [1045.83996196],
       [1220.10850237],
       [ 798.6799921 ],
       [ 399.01634709],
       [ 530.26085263],
       [-111.30767333],
       [ 540.48442601],
       [ 756.86800736],
       [ 333.41100629],
       [ 727.41145817],
       [ 649.33812135],
       [ 666.26170942],
       [ 192.85749955],
       [1190.41649923],
       [ 587.80261873],
       [1296.48723489],
       [ 367.61147203],
       [ 326.88801256],
       [1360.03206926],
       [1128.51984269],
       [ 408.02945778],
       [ 149.72398763],
       [1143.60139983],
       [ 769.53515161],
       [ 700.34526903],
       [-104.725

In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)